# Environment setting

In [ ]:
kaggle = True

In [ ]:
!pip install -q /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
!pip install -q /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
!pip install -q /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
!pip install -q /kaggle/input/pip-install-lifelines/formulaic-1.0.2-py3-none-any.whl
!pip install -q /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl
!pip install -q /kaggle/input/download-lightning-and-pytorch-tabular/pytorch_lightning-2.4.0-py3-none-any.whl
!pip install -q /kaggle/input/download-lightning-and-pytorch-tabular/scikit_learn-1.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install -q /kaggle/input/download-lightning-and-pytorch-tabular/torchmetrics-1.5.2-py3-none-any.whl
!pip install -q /kaggle/input/download-lightning-and-pytorch-tabular/pytorch_tabnet-4.1.0-py3-none-any.whl
!pip install -q /kaggle/input/download-lightning-and-pytorch-tabular/einops-0.7.0-py3-none-any.whl
!pip install -q /kaggle/input/download-lightning-and-pytorch-tabular/pytorch_tabular-1.1.1-py2.py3-none-any.whl

!pip install /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
!pip install /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/formulaic-1.0.2-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl

# TabM

## Prepare

### prepare data

In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from torch.utils.data import TensorDataset
from warnings import filterwarnings

filterwarnings('ignore')


def get_X_cat(df, cat_cols, transformers=None):
    """
    Apply a specific categorical data transformer or a LabelEncoder if None.
    """
    if transformers is None:
        transformers = [LabelEncoder().fit(df[col]) for col in cat_cols]
    return transformers, np.array(
        [transformer.transform(df[col]) for col, transformer in zip(cat_cols, transformers)]
    ).T


def preprocess_data(train, val):
    """
    Standardize numerical variables and transform (Label-encode) categoricals.
    Fill NA values with mean for numerical.
    Create torch dataloaders to prepare data for training and evaluation.
    """
    X_cat_train, X_cat_val, numerical, transformers = get_categoricals(train, val)
    scaler = StandardScaler()
    imp = SimpleImputer(missing_values=np.nan, strategy='mean', add_indicator=True)
    X_num_train = imp.fit_transform(train[numerical])
    X_num_train = scaler.fit_transform(X_num_train)
    X_num_val = imp.transform(val[numerical])
    X_num_val = scaler.transform(X_num_val)
    dl_train = init_dl(X_cat_train, X_num_train, train, training=True)
    dl_val = init_dl(X_cat_val, X_num_val, val)
    return X_cat_val, X_num_train, X_num_val, dl_train, dl_val, transformers


def get_categoricals(train, val):
    """
    Remove constant categorical columns and transform them using LabelEncoder.
    Return the label-transformers for each categorical column, categorical dataframes and numerical columns.
    """
    categorical_cols, numerical = get_feature_types(train)
    remove = []
    for col in categorical_cols:
        if train[col].nunique() == 1:
            remove.append(col)
        ind = ~val[col].isin(train[col])
        if ind.any():
            val.loc[ind, col] = np.nan
    categorical_cols = [col for col in categorical_cols if col not in remove]
    transformers, X_cat_train = get_X_cat(train, categorical_cols)
    _, X_cat_val = get_X_cat(val, categorical_cols, transformers)
    return X_cat_train, X_cat_val, numerical, transformers


def init_dl(X_cat, X_num, df, training=False):
    """
    Initialize data loaders with 4 dimensions : categorical dataframe, numerical dataframe and target values (efs and efs_time).
    Notice that efs_time is log-transformed.
    Fix batch size to 2048 and return dataloader for training or validation depending on training value.
    """
    ds_train = TensorDataset(
        torch.tensor(X_cat, dtype=torch.long),
        torch.tensor(X_num, dtype=torch.float32),
        torch.tensor(df.efs_time.values, dtype=torch.float32).log(),
        torch.tensor(df.efs.values, dtype=torch.long)
    )
    bs = 2048
    dl_train = torch.utils.data.DataLoader(ds_train, batch_size=bs, pin_memory=True, shuffle=training)
    return dl_train


def get_feature_types(train):
    """
    Utility function to return categorical and numerical column names.
    """
    categorical_cols = [col for i, col in enumerate(train.columns) if ((train[col].dtype == "object") | (2 < train[col].nunique() < 25))]
    RMV = ["ID", "efs", "efs_time", "y"]
    FEATURES = [c for c in train.columns if not c in RMV]
    numerical = [i for i in FEATURES if i not in categorical_cols]
    return categorical_cols, numerical


def add_features(df):
    """
    Create some new features to help the model focus on specific patterns.
    """
    # sex_match = df.sex_match.astype(str)
    # sex_match = sex_match.str.split("-").str[0] == sex_match.str.split("-").str[1]
    # df['sex_match_bool'] = sex_match
    # df.loc[df.sex_match.isna(), 'sex_match_bool'] = np.nan
    # df['big_age'] = df.age_at_hct > 16
    # df.loc[df.year_hct == 2019, 'year_hct'] = 2020
    df['is_cyto_score_same'] = (df['cyto_score'] == df['cyto_score_detail']).astype(int)
    # df['strange_age'] = df.age_at_hct == 0.044
    # df['age_bin'] = pd.cut(df.age_at_hct, [0, 0.0441, 16, 30, 50, 100])
    # df['age_ts'] = df.age_at_hct / df.donor_age
    df['year_hct'] -= 2000
    
    return df


def load_data():
    """
    Load data and add features.
    """
    if kaggle:
        test = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/test.csv")
        train = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/train.csv")
    else:
        test = pd.read_csv("../data/test.csv")
        train = pd.read_csv("../data/train.csv")
        
    test = add_features(test)
    train = add_features(train)
    print("Test shape:", test.shape)
    print("Train shape:", train.shape)
    return test, train

### prepare model

In [ ]:
import functools
from typing import List

import pytorch_lightning as pl
import numpy as np
import torch
from lifelines.utils import concordance_index
from pytorch_lightning.cli import ReduceLROnPlateau
from pytorch_tabular.models.common.layers import ODST
from torch import nn
from pytorch_lightning.utilities import grad_norm


class CatEmbeddings(nn.Module):
    """
    Embedding module for the categorical dataframe.
    """
    def __init__(
        self,
        projection_dim: int,
        categorical_cardinality: List[int],
        embedding_dim: int
    ):
        """
        projection_dim: The dimension of the final output after projecting the concatenated embeddings into a lower-dimensional space.
        categorical_cardinality: A list where each element represents the number of unique categories (cardinality) in each categorical feature.
        embedding_dim: The size of the embedding space for each categorical feature.
        self.embeddings: list of embedding layers for each categorical feature.
        self.projection: sequential neural network that goes from the embedding to the output projection dimension with GELU activation.
        """
        super(CatEmbeddings, self).__init__()
        self.embeddings = nn.ModuleList([
            nn.Embedding(cardinality, embedding_dim)
            for cardinality in categorical_cardinality
        ])
        self.projection = nn.Sequential(
            nn.Linear(embedding_dim * len(categorical_cardinality), projection_dim),
            nn.GELU(),
            nn.Linear(projection_dim, projection_dim)
        )

    def forward(self, x_cat):
        """
        Apply the projection on concatened embeddings that contains all categorical features.
        """
        x_cat = [embedding(x_cat[:, i]) for i, embedding in enumerate(self.embeddings)]
        x_cat = torch.cat(x_cat, dim=1)
        return self.projection(x_cat)


class NN(nn.Module):
    """
    Train a model on both categorical embeddings and numerical data.
    """
    def __init__(
            self,
            continuous_dim: int,
            categorical_cardinality: List[int],
            embedding_dim: int,
            projection_dim: int,
            hidden_dim: int,
            dropout: float = 0
    ):
        """
        continuous_dim: The number of continuous features.
        categorical_cardinality: A list of integers representing the number of unique categories in each categorical feature.
        embedding_dim: The dimensionality of the embedding space for each categorical feature.
        projection_dim: The size of the projected output space for the categorical embeddings.
        hidden_dim: The number of neurons in the hidden layer of the MLP.
        dropout: The dropout rate applied in the network.
        self.embeddings: previous embeddings for categorical data.
        self.mlp: defines an MLP model with an ODST layer followed by batch normalization and dropout.
        self.out: linear output layer that maps the output of the MLP to a single value
        self.dropout: defines dropout
        Weights initialization with xavier normal algorithm and biases with zeros.
        """
        super(NN, self).__init__()
        self.embeddings = CatEmbeddings(projection_dim, categorical_cardinality, embedding_dim)
        self.mlp = nn.Sequential(
            ODST(projection_dim + continuous_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(dropout)
        )
        self.out = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(dropout)

        # initialize weights
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, x_cat, x_cont):
        """
        Create embedding layers for categorical data, concatenate with continous variables.
        Add dropout and goes through MLP and return raw output and 1-dimensional output as well.
        """
        x = self.embeddings(x_cat)
        x = torch.cat([x, x_cont], dim=1)
        x = self.dropout(x)
        x = self.mlp(x)
        return self.out(x), x


@functools.lru_cache
def combinations(N):
    """
    calculates all possible 2-combinations (pairs) of a tensor of indices from 0 to N-1, 
    and caches the result using functools.lru_cache for optimization
    """
    ind = torch.arange(N)
    comb = torch.combinations(ind, r=2)
    return comb.cuda()


class LitNN(pl.LightningModule):
    """
    Main Model creation and losses definition to fully train the model.
    """
    def __init__(
            self,
            continuous_dim: int,
            categorical_cardinality: List[int],
            embedding_dim: int,
            projection_dim: int,
            hidden_dim: int,
            lr: float = 1e-3,
            dropout: float = 0.2,
            weight_decay: float = 1e-3,
            aux_weight: float = 0.1,
            margin: float = 0.5,
            race_index: int = 0
    ):
        """
        continuous_dim: The number of continuous input features.
        categorical_cardinality: A list of integers, where each element corresponds to the number of unique categories for each categorical feature.
        embedding_dim: The dimension of the embeddings for the categorical features.
        projection_dim: The dimension of the projected space after embedding concatenation.
        hidden_dim: The size of the hidden layers in the feedforward network (MLP).
        lr: The learning rate for the optimizer.
        dropout: Dropout probability to avoid overfitting.
        weight_decay: The L2 regularization term for the optimizer.
        aux_weight: Weight used for auxiliary tasks.
        margin: Margin used in some loss functions.
        race_index: An index that refer to race_group in the input data.
        """
        super(LitNN, self).__init__()
        self.save_hyperparameters()

        # Creates an instance of the NN model defined above
        self.model = NN(
            continuous_dim=self.hparams.continuous_dim,
            categorical_cardinality=self.hparams.categorical_cardinality,
            embedding_dim=self.hparams.embedding_dim,
            projection_dim=self.hparams.projection_dim,
            hidden_dim=self.hparams.hidden_dim,
            dropout=self.hparams.dropout
        )
        self.targets = []

        # Defines a small feedforward neural network that performs an auxiliary task with 1-dimensional output
        self.aux_cls = nn.Sequential(
            nn.Linear(self.hparams.hidden_dim, self.hparams.hidden_dim // 3),
            nn.GELU(),
            nn.Linear(self.hparams.hidden_dim // 3, 1)
        )

    def on_before_optimizer_step(self, optimizer):
        """
        Compute the 2-norm for each layer
        If using mixed precision, the gradients are already unscaled here
        """
        norms = grad_norm(self.model, norm_type=2)
        self.log_dict(norms)

    def forward(self, x_cat, x_cont):
        """
        Forward pass that outputs the 1-dimensional prediction and the embeddings (raw output)
        """
        x, emb = self.model(x_cat, x_cont)
        return x.squeeze(1), emb

    def training_step(self, batch, batch_idx):
        """
        defines how the model processes each batch of data during training.
        A batch is a combination of : categorical data, continuous data, efs_time (y) and efs event.
        y_hat is the efs_time prediction on all data and aux_pred is auxiliary prediction on embeddings.
        Calculates loss and race_group loss on full data.
        Auxiliary loss is calculated with an event mask, ignoring efs=0 predictions and taking the average.
        Returns loss and aux_loss multiplied by weight defined above.
        """
        x_cat, x_cont, y, efs = batch
        y_hat, emb = self(x_cat, x_cont)
        aux_pred = self.aux_cls(emb).squeeze(1)
        loss, race_loss = self.get_full_loss(efs, x_cat, y, y_hat)
        aux_loss = nn.functional.mse_loss(aux_pred, y, reduction='none')
        aux_mask = efs == 1
        aux_loss = (aux_loss * aux_mask).sum() / aux_mask.sum()
        self.log("train_loss", loss, on_epoch=True, prog_bar=True, logger=True)
        self.log("race_loss", race_loss, on_epoch=True, prog_bar=True, logger=True, on_step=False)
        self.log("aux_loss", aux_loss, on_epoch=True, prog_bar=True, logger=True, on_step=False)
        return loss + aux_loss * self.hparams.aux_weight

    def get_full_loss(self, efs, x_cat, y, y_hat):
        """
        Output loss and race_group loss.
        """
        loss = self.calc_loss(y, y_hat, efs)
        race_loss = self.get_race_losses(efs, x_cat, y, y_hat)
        loss += 0.1 * race_loss
        return loss, race_loss

    def get_race_losses(self, efs, x_cat, y, y_hat):
        """
        Calculate loss for each race_group based on deviation/variance.
        """
        races = torch.unique(x_cat[:, self.hparams.race_index])
        race_losses = []
        for race in races:
            ind = x_cat[:, self.hparams.race_index] == race
            race_losses.append(self.calc_loss(y[ind], y_hat[ind], efs[ind]))
        race_loss = sum(race_losses) / len(race_losses)
        races_loss_std = sum((r - race_loss)**2 for r in race_losses) / len(race_losses)
        return torch.sqrt(races_loss_std)

    def calc_loss(self, y, y_hat, efs):
        """
        Most important part of the model : loss function used for training.
        We face survival data with event indicators along with time-to-event.

        This function computes the main loss by the following the steps :
        * create all data pairs with "combinations" function (= all "two subjects" combinations)
        * make sure that we have at least 1 event in each pair
        * convert y to +1 or -1 depending on the correct ranking
        * loss is computed using a margin-based hinge loss
        * mask is applied to ensure only valid pairs are being used (censored data can't be ranked with event in some cases)
        * average loss on all pairs is returned
        """
        N = y.shape[0]
        comb = combinations(N)
        comb = comb[(efs[comb[:, 0]] == 1) | (efs[comb[:, 1]] == 1)]
        pred_left = y_hat[comb[:, 0]]
        pred_right = y_hat[comb[:, 1]]
        y_left = y[comb[:, 0]]
        y_right = y[comb[:, 1]]
        y = 2 * (y_left > y_right).int() - 1
        loss = nn.functional.relu(-y * (pred_left - pred_right) + self.hparams.margin)
        mask = self.get_mask(comb, efs, y_left, y_right)
        loss = (loss.double() * (mask.double())).sum() / mask.sum()
        return loss

    def get_mask(self, comb, efs, y_left, y_right):
        """
        Defines all invalid comparisons :
        * Case 1: "Left outlived Right" but Right is censored
        * Case 2: "Right outlived Left" but Left is censored
        Masks for case 1 and case 2 are combined using |= operator and inverted using ~ to create a "valid pair mask"
        """
        left_outlived = y_left >= y_right
        left_1_right_0 = (efs[comb[:, 0]] == 1) & (efs[comb[:, 1]] == 0)
        mask2 = (left_outlived & left_1_right_0)
        right_outlived = y_right >= y_left
        right_1_left_0 = (efs[comb[:, 1]] == 1) & (efs[comb[:, 0]] == 0)
        mask2 |= (right_outlived & right_1_left_0)
        mask2 = ~mask2
        mask = mask2
        return mask

    def validation_step(self, batch, batch_idx):
        """
        This method defines how the model processes each batch during validation
        """
        x_cat, x_cont, y, efs = batch
        y_hat, emb = self(x_cat, x_cont)
        loss, race_loss = self.get_full_loss(efs, x_cat, y, y_hat)
        self.targets.append([y, y_hat.detach(), efs, x_cat[:, self.hparams.race_index]])
        self.log("val_loss", loss, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def on_validation_epoch_end(self):
        """
        At the end of the validation epoch, it computes and logs the concordance index
        """
        cindex, metric = self._calc_cindex()
        self.log("cindex", metric, on_epoch=True, prog_bar=True, logger=True)
        self.log("cindex_simple", cindex, on_epoch=True, prog_bar=True, logger=True)
        self.targets.clear()

    def _calc_cindex(self):
        """
        Calculate c-index accounting for each race_group or global.
        """
        y = torch.cat([t[0] for t in self.targets]).cpu().numpy()
        y_hat = torch.cat([t[1] for t in self.targets]).cpu().numpy()
        efs = torch.cat([t[2] for t in self.targets]).cpu().numpy()
        races = torch.cat([t[3] for t in self.targets]).cpu().numpy()
        metric = self._metric(efs, races, y, y_hat)
        cindex = concordance_index(y, y_hat, efs)
        return cindex, metric

    def _metric(self, efs, races, y, y_hat):
        """
        Calculate c-index accounting for each race_group
        """
        metric_list = []
        for race in np.unique(races):
            y_ = y[races == race]
            y_hat_ = y_hat[races == race]
            efs_ = efs[races == race]
            metric_list.append(concordance_index(y_, y_hat_, efs_))
        metric = float(np.mean(metric_list) - np.sqrt(np.var(metric_list)))
        return metric

    def test_step(self, batch, batch_idx):
        """
        Same as training step but to log test data
        """
        x_cat, x_cont, y, efs = batch
        y_hat, emb = self(x_cat, x_cont)
        loss, race_loss = self.get_full_loss(efs, x_cat, y, y_hat)
        self.targets.append([y, y_hat.detach(), efs, x_cat[:, self.hparams.race_index]])
        self.log("test_loss", loss)
        return loss

    def on_test_epoch_end(self) -> None:
        """
        At the end of the test epoch, calculates and logs the concordance index for the test set
        """
        cindex, metric = self._calc_cindex()
        self.log("test_cindex", metric, on_epoch=True, prog_bar=True, logger=True)
        self.log("test_cindex_simple", cindex, on_epoch=True, prog_bar=True, logger=True)
        self.targets.clear()


    def configure_optimizers(self):
        """
        configures the optimizer and learning rate scheduler:
        * Optimizer: Adam optimizer with weight decay (L2 regularization).
        * Scheduler: Cosine Annealing scheduler, which adjusts the learning rate according to a cosine curve.
        """
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.lr, weight_decay=self.hparams.weight_decay)
        scheduler_config = {
            "scheduler": torch.optim.lr_scheduler.CosineAnnealingLR(
                optimizer,
                T_max=45,
                eta_min=6e-3
            ),
            "interval": "epoch",
            "frequency": 1,
            "strict": False,
        }

        return {"optimizer": optimizer, "lr_scheduler": scheduler_config}

## Train and Predict

In [ ]:
# import json
import pytorch_lightning as pl
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import torch
from pytorch_lightning.callbacks import LearningRateMonitor, TQDMProgressBar
from pytorch_lightning.callbacks import StochasticWeightAveraging
from sklearn.model_selection import KFold, StratifiedKFold

pl.seed_everything(42)

def main(hparams):
    """
    Main function to train the model.
    The steps are as following :
    * load data and fill efs and efs time for test data with 1
    * initialize pred array with 0
    * get categorical and numerical columns
    * split the train data on the stratified criterion : race_group * newborns yes/no
    * preprocess the fold data (create dataloaders)
    * train the model and create final submission output
    """
    test, train_original = load_data()
    test['efs_time'] = 1
    test['efs'] = 1
    test_pred = np.zeros(test.shape[0])
    val_pred = np.zeros(train_original.shape[0])


    categorical_cols, numerical = get_feature_types(train_original)
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    for i, (train_index, test_index) in enumerate(
        kf.split(
            train_original, train_original.race_group.astype(str) + (train_original.age_at_hct == 0.044).astype(str)
        )
    ):
        tt = train_original.copy()
        train = tt.iloc[train_index]
        val = tt.iloc[test_index]
        print('+++++++++',val.shape)
        X_cat_val, X_num_train, X_num_val, dl_train, dl_val, transformers = preprocess_data(train, val)
        
        
        model = train_final(X_num_train, dl_train, dl_val, transformers, categorical_cols=categorical_cols)

        ### eval on valid
        pred, _ = model.cuda().eval()(
            torch.tensor(X_cat_val, dtype=torch.long).cuda(),
            torch.tensor(X_num_val, dtype=torch.float32).cuda()
        )

        
        val_pred[test_index] = pred.detach().cpu().numpy()

        # Create submission
        train = tt.iloc[train_index]
        X_cat_val, X_num_train, X_num_val, dl_train, dl_val, transformers = preprocess_data(train, test)

        pred, _ = model.cuda().eval()(
            torch.tensor(X_cat_val, dtype=torch.long).cuda(),
            torch.tensor(X_num_val, dtype=torch.float32).cuda()
        )

        test_pred += pred.detach().cpu().numpy()
        

    return val_pred , test_pred



def train_final(X_num_train, dl_train, dl_val, transformers, hparams=None, categorical_cols=None):
    """
    Defines model hyperparameters and fit the model.
    """
    if hparams is None:
        hparams = {
            "embedding_dim": 16,
            "projection_dim": 112,
            "hidden_dim": 56,
            "lr": 0.06464861983337984,
            "dropout": 0.05463240181423116,
            "aux_weight": 0.26545778308743806,
            "margin": 0.2588153271003354,
            "weight_decay": 0.0002773544957610778
        }
    model = LitNN(
        continuous_dim=X_num_train.shape[1],
        categorical_cardinality=[len(t.classes_) for t in transformers],
        race_index=categorical_cols.index("race_group"),
        **hparams
    )
    checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor="val_loss", save_top_k=1)
    trainer = pl.Trainer(
        accelerator='cuda',
        max_epochs=55,
        log_every_n_steps=6,
        callbacks=[
            checkpoint_callback,
            LearningRateMonitor(logging_interval='epoch'),
            TQDMProgressBar(),
            StochasticWeightAveraging(swa_lrs=1e-5, swa_epoch_start=40, annealing_epochs=15)
        ],
    )
    trainer.fit(model, dl_train)
    trainer.test(model, dl_val)
    return model.eval()


hparams = None
tabm_oof_preds, tabm_preds = main(hparams)
print("done")

# NN pairwise model

## Prepare

In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from torch.utils.data import TensorDataset


def get_X_cat(df, cat_cols, transformers=None):
    """
    Apply a specific categorical data transformer or a LabelEncoder if None.
    """
    if transformers is None:
        transformers = [LabelEncoder().fit(df[col]) for col in cat_cols]
    return transformers, np.array(
        [transformer.transform(df[col]) for col, transformer in zip(cat_cols, transformers)]
    ).T


def preprocess_data(train, val):
    """
    Standardize numerical variables and transform (Label-encode) categoricals.
    Fill NA values with mean for numerical.
    Create torch dataloaders to prepare data for training and evaluation.
    """
    X_cat_train, X_cat_val, numerical, transformers = get_categoricals(train, val)
    scaler = StandardScaler()
    imp = SimpleImputer(missing_values=np.nan, strategy='mean', add_indicator=True)
    X_num_train = imp.fit_transform(train[numerical])
    X_num_train = scaler.fit_transform(X_num_train)
    X_num_val = imp.transform(val[numerical])
    X_num_val = scaler.transform(X_num_val)
    dl_train = init_dl(X_cat_train, X_num_train, train, training=True)
    dl_val = init_dl(X_cat_val, X_num_val, val)
    return X_cat_val, X_num_train, X_num_val, dl_train, dl_val, transformers


def get_categoricals(train, val):
    """
    Remove constant categorical columns and transform them using LabelEncoder.
    Return the label-transformers for each categorical column, categorical dataframes and numerical columns.
    """
    categorical_cols, numerical = get_feature_types(train)
    remove = []
    for col in categorical_cols:
        if train[col].nunique() == 1:
            remove.append(col)
        ind = ~val[col].isin(train[col])
        if ind.any():
            val.loc[ind, col] = np.nan
    categorical_cols = [col for col in categorical_cols if col not in remove]
    transformers, X_cat_train = get_X_cat(train, categorical_cols)
    _, X_cat_val = get_X_cat(val, categorical_cols, transformers)
    return X_cat_train, X_cat_val, numerical, transformers


def init_dl(X_cat, X_num, df, training=False):
    """
    Initialize data loaders with 4 dimensions : categorical dataframe, numerical dataframe and target values (efs and efs_time).
    Notice that efs_time is log-transformed.
    Fix batch size to 2048 and return dataloader for training or validation depending on training value.
    """
    ds_train = TensorDataset(
        torch.tensor(X_cat, dtype=torch.long),
        torch.tensor(X_num, dtype=torch.float32),
        torch.tensor(df.efs_time.values, dtype=torch.float32).log(),
        torch.tensor(df.efs.values, dtype=torch.long)
    )
    bs = 2048
    dl_train = torch.utils.data.DataLoader(ds_train, batch_size=bs, pin_memory=True, shuffle=training)
    return dl_train


def get_feature_types(train):
    """
    Utility function to return categorical and numerical column names.
    """
    categorical_cols = [col for i, col in enumerate(train.columns) if ((train[col].dtype == "object") | (2 < train[col].nunique() < 25))]
    RMV = ["ID", "efs", "efs_time", "y"]
    FEATURES = [c for c in train.columns if not c in RMV]
    numerical = [i for i in FEATURES if i not in categorical_cols]
    return categorical_cols, numerical


def add_features(df):
    """
    Create some new features to help the model focus on specific patterns.
    """
    sex_match = df.sex_match.astype(str)
    sex_match = sex_match.str.split("-").str[0] == sex_match.str.split("-").str[1]
    df['sex_match_bool'] = sex_match
    df.loc[df.sex_match.isna(), 'sex_match_bool'] = np.nan
    df['big_age'] = df.age_at_hct > 16
    df.loc[df.year_hct == 2019, 'year_hct'] = 2020
    df['is_cyto_score_same'] = (df['cyto_score'] == df['cyto_score_detail']).astype(int)
    df['strange_age'] = df.age_at_hct == 0.044
    df['age_bin'] = pd.cut(df.age_at_hct, [0, 0.0441, 16, 30, 50, 100])
    df['age_ts'] = df.age_at_hct / df.donor_age
    df['year_hct'] -= 2000
    
    return df


def load_data():
    """
    Load data and add features.
    """
    if kaggle:
        test = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/test.csv")
    else:
        test = pd.read_csv("../data/test.csv")
        
    test = add_features(test)
    print("Test shape:", test.shape)
    if kaggle:
        train = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/train.csv")
    else:
        train = pd.read_csv("../data/train.csv")
    train = add_features(train)
    print("Train shape:", train.shape)
    return test, train

## Model

In [ ]:
import functools
from typing import List

import pytorch_lightning as pl
import numpy as np
import torch
from lifelines.utils import concordance_index
from pytorch_lightning.cli import ReduceLROnPlateau
from pytorch_tabular.models.common.layers import ODST
from torch import nn
from pytorch_lightning.utilities import grad_norm


class CatEmbeddings(nn.Module):
    """
    Embedding module for the categorical dataframe.
    """
    def __init__(
        self,
        projection_dim: int,
        categorical_cardinality: List[int],
        embedding_dim: int
    ):
        """
        projection_dim: The dimension of the final output after projecting the concatenated embeddings into a lower-dimensional space.
        categorical_cardinality: A list where each element represents the number of unique categories (cardinality) in each categorical feature.
        embedding_dim: The size of the embedding space for each categorical feature.
        self.embeddings: list of embedding layers for each categorical feature.
        self.projection: sequential neural network that goes from the embedding to the output projection dimension with GELU activation.
        """
        super(CatEmbeddings, self).__init__()
        self.embeddings = nn.ModuleList([
            nn.Embedding(cardinality, embedding_dim)
            for cardinality in categorical_cardinality
        ])
        self.projection = nn.Sequential(
            nn.Linear(embedding_dim * len(categorical_cardinality), projection_dim),
            nn.GELU(),
            nn.Linear(projection_dim, projection_dim)
        )

    def forward(self, x_cat):
        """
        Apply the projection on concatened embeddings that contains all categorical features.
        """
        x_cat = [embedding(x_cat[:, i]) for i, embedding in enumerate(self.embeddings)]
        x_cat = torch.cat(x_cat, dim=1)
        return self.projection(x_cat)


class NN(nn.Module):
    """
    Train a model on both categorical embeddings and numerical data.
    """
    def __init__(
            self,
            continuous_dim: int,
            categorical_cardinality: List[int],
            embedding_dim: int,
            projection_dim: int,
            hidden_dim: int,
            dropout: float = 0
    ):
        """
        continuous_dim: The number of continuous features.
        categorical_cardinality: A list of integers representing the number of unique categories in each categorical feature.
        embedding_dim: The dimensionality of the embedding space for each categorical feature.
        projection_dim: The size of the projected output space for the categorical embeddings.
        hidden_dim: The number of neurons in the hidden layer of the MLP.
        dropout: The dropout rate applied in the network.
        self.embeddings: previous embeddings for categorical data.
        self.mlp: defines an MLP model with an ODST layer followed by batch normalization and dropout.
        self.out: linear output layer that maps the output of the MLP to a single value
        self.dropout: defines dropout
        Weights initialization with xavier normal algorithm and biases with zeros.
        """
        super(NN, self).__init__()
        self.embeddings = CatEmbeddings(projection_dim, categorical_cardinality, embedding_dim)
        self.mlp = nn.Sequential(
            ODST(projection_dim + continuous_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(dropout)
        )
        self.out = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(dropout)

        # initialize weights
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, x_cat, x_cont):
        """
        Create embedding layers for categorical data, concatenate with continous variables.
        Add dropout and goes through MLP and return raw output and 1-dimensional output as well.
        """
        x = self.embeddings(x_cat)
        x = torch.cat([x, x_cont], dim=1)
        x = self.dropout(x)
        x = self.mlp(x)
        return self.out(x), x


@functools.lru_cache
def combinations(N):
    """
    calculates all possible 2-combinations (pairs) of a tensor of indices from 0 to N-1, 
    and caches the result using functools.lru_cache for optimization
    """
    ind = torch.arange(N)
    comb = torch.combinations(ind, r=2)
    return comb.cuda()


class LitNN(pl.LightningModule):
    """
    Main Model creation and losses definition to fully train the model.
    """
    def __init__(
            self,
            continuous_dim: int,
            categorical_cardinality: List[int],
            embedding_dim: int,
            projection_dim: int,
            hidden_dim: int,
            lr: float = 1e-3,
            dropout: float = 0.2,
            weight_decay: float = 1e-3,
            aux_weight: float = 0.1,
            margin: float = 0.5,
            race_index: int = 0
    ):
        """
        continuous_dim: The number of continuous input features.
        categorical_cardinality: A list of integers, where each element corresponds to the number of unique categories for each categorical feature.
        embedding_dim: The dimension of the embeddings for the categorical features.
        projection_dim: The dimension of the projected space after embedding concatenation.
        hidden_dim: The size of the hidden layers in the feedforward network (MLP).
        lr: The learning rate for the optimizer.
        dropout: Dropout probability to avoid overfitting.
        weight_decay: The L2 regularization term for the optimizer.
        aux_weight: Weight used for auxiliary tasks.
        margin: Margin used in some loss functions.
        race_index: An index that refer to race_group in the input data.
        """
        super(LitNN, self).__init__()
        self.save_hyperparameters()

        # Creates an instance of the NN model defined above
        self.model = NN(
            continuous_dim=self.hparams.continuous_dim,
            categorical_cardinality=self.hparams.categorical_cardinality,
            embedding_dim=self.hparams.embedding_dim,
            projection_dim=self.hparams.projection_dim,
            hidden_dim=self.hparams.hidden_dim,
            dropout=self.hparams.dropout
        )
        self.targets = []

        # Defines a small feedforward neural network that performs an auxiliary task with 1-dimensional output
        self.aux_cls = nn.Sequential(
            nn.Linear(self.hparams.hidden_dim, self.hparams.hidden_dim // 3),
            nn.GELU(),
            nn.Linear(self.hparams.hidden_dim // 3, 1)
        )

    def on_before_optimizer_step(self, optimizer):
        """
        Compute the 2-norm for each layer
        If using mixed precision, the gradients are already unscaled here
        """
        norms = grad_norm(self.model, norm_type=2)
        self.log_dict(norms)

    def forward(self, x_cat, x_cont):
        """
        Forward pass that outputs the 1-dimensional prediction and the embeddings (raw output)
        """
        x, emb = self.model(x_cat, x_cont)
        return x.squeeze(1), emb

    def training_step(self, batch, batch_idx):
        """
        defines how the model processes each batch of data during training.
        A batch is a combination of : categorical data, continuous data, efs_time (y) and efs event.
        y_hat is the efs_time prediction on all data and aux_pred is auxiliary prediction on embeddings.
        Calculates loss and race_group loss on full data.
        Auxiliary loss is calculated with an event mask, ignoring efs=0 predictions and taking the average.
        Returns loss and aux_loss multiplied by weight defined above.
        """
        x_cat, x_cont, y, efs = batch
        y_hat, emb = self(x_cat, x_cont)
        aux_pred = self.aux_cls(emb).squeeze(1)
        loss, race_loss = self.get_full_loss(efs, x_cat, y, y_hat)
        aux_loss = nn.functional.mse_loss(aux_pred, y, reduction='none')
        aux_mask = efs == 1
        aux_loss = (aux_loss * aux_mask).sum() / aux_mask.sum()
        self.log("train_loss", loss, on_epoch=True, prog_bar=True, logger=True)
        self.log("race_loss", race_loss, on_epoch=True, prog_bar=True, logger=True, on_step=False)
        self.log("aux_loss", aux_loss, on_epoch=True, prog_bar=True, logger=True, on_step=False)
        return loss + aux_loss * self.hparams.aux_weight

    def get_full_loss(self, efs, x_cat, y, y_hat):
        """
        Output loss and race_group loss.
        """
        loss = self.calc_loss(y, y_hat, efs)
        race_loss = self.get_race_losses(efs, x_cat, y, y_hat)
        loss += 0.1 * race_loss
        return loss, race_loss

    def get_race_losses(self, efs, x_cat, y, y_hat):
        """
        Calculate loss for each race_group based on deviation/variance.
        """
        races = torch.unique(x_cat[:, self.hparams.race_index])
        race_losses = []
        for race in races:
            ind = x_cat[:, self.hparams.race_index] == race
            race_losses.append(self.calc_loss(y[ind], y_hat[ind], efs[ind]))
        race_loss = sum(race_losses) / len(race_losses)
        races_loss_std = sum((r - race_loss)**2 for r in race_losses) / len(race_losses)
        return torch.sqrt(races_loss_std)

    def calc_loss(self, y, y_hat, efs):
        """
        Most important part of the model : loss function used for training.
        We face survival data with event indicators along with time-to-event.

        This function computes the main loss by the following the steps :
        * create all data pairs with "combinations" function (= all "two subjects" combinations)
        * make sure that we have at least 1 event in each pair
        * convert y to +1 or -1 depending on the correct ranking
        * loss is computed using a margin-based hinge loss
        * mask is applied to ensure only valid pairs are being used (censored data can't be ranked with event in some cases)
        * average loss on all pairs is returned
        """
        N = y.shape[0]
        comb = combinations(N)
        comb = comb[(efs[comb[:, 0]] == 1) | (efs[comb[:, 1]] == 1)]
        pred_left = y_hat[comb[:, 0]]
        pred_right = y_hat[comb[:, 1]]
        y_left = y[comb[:, 0]]
        y_right = y[comb[:, 1]]
        y = 2 * (y_left > y_right).int() - 1
        loss = nn.functional.relu(-y * (pred_left - pred_right) + self.hparams.margin)
        mask = self.get_mask(comb, efs, y_left, y_right)
        loss = (loss.double() * (mask.double())).sum() / mask.sum()
        return loss

    def get_mask(self, comb, efs, y_left, y_right):
        """
        Defines all invalid comparisons :
        * Case 1: "Left outlived Right" but Right is censored
        * Case 2: "Right outlived Left" but Left is censored
        Masks for case 1 and case 2 are combined using |= operator and inverted using ~ to create a "valid pair mask"
        """
        left_outlived = y_left >= y_right
        left_1_right_0 = (efs[comb[:, 0]] == 1) & (efs[comb[:, 1]] == 0)
        mask2 = (left_outlived & left_1_right_0)
        right_outlived = y_right >= y_left
        right_1_left_0 = (efs[comb[:, 1]] == 1) & (efs[comb[:, 0]] == 0)
        mask2 |= (right_outlived & right_1_left_0)
        mask2 = ~mask2
        mask = mask2
        return mask

    def validation_step(self, batch, batch_idx):
        """
        This method defines how the model processes each batch during validation
        """
        x_cat, x_cont, y, efs = batch
        y_hat, emb = self(x_cat, x_cont)
        loss, race_loss = self.get_full_loss(efs, x_cat, y, y_hat)
        self.targets.append([y, y_hat.detach(), efs, x_cat[:, self.hparams.race_index]])
        self.log("val_loss", loss, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def on_validation_epoch_end(self):
        """
        At the end of the validation epoch, it computes and logs the concordance index
        """
        cindex, metric = self._calc_cindex()
        self.log("cindex", metric, on_epoch=True, prog_bar=True, logger=True)
        self.log("cindex_simple", cindex, on_epoch=True, prog_bar=True, logger=True)
        self.targets.clear()

    def _calc_cindex(self):
        """
        Calculate c-index accounting for each race_group or global.
        """
        y = torch.cat([t[0] for t in self.targets]).cpu().numpy()
        y_hat = torch.cat([t[1] for t in self.targets]).cpu().numpy()
        efs = torch.cat([t[2] for t in self.targets]).cpu().numpy()
        races = torch.cat([t[3] for t in self.targets]).cpu().numpy()
        metric = self._metric(efs, races, y, y_hat)
        cindex = concordance_index(y, y_hat, efs)
        return cindex, metric

    def _metric(self, efs, races, y, y_hat):
        """
        Calculate c-index accounting for each race_group
        """
        metric_list = []
        for race in np.unique(races):
            y_ = y[races == race]
            y_hat_ = y_hat[races == race]
            efs_ = efs[races == race]
            metric_list.append(concordance_index(y_, y_hat_, efs_))
        metric = float(np.mean(metric_list) - np.sqrt(np.var(metric_list)))
        return metric

    def test_step(self, batch, batch_idx):
        """
        Same as training step but to log test data
        """
        x_cat, x_cont, y, efs = batch
        y_hat, emb = self(x_cat, x_cont)
        loss, race_loss = self.get_full_loss(efs, x_cat, y, y_hat)
        self.targets.append([y, y_hat.detach(), efs, x_cat[:, self.hparams.race_index]])
        self.log("test_loss", loss)
        return loss

    def on_test_epoch_end(self) -> None:
        """
        At the end of the test epoch, calculates and logs the concordance index for the test set
        """
        cindex, metric = self._calc_cindex()
        self.log("test_cindex", metric, on_epoch=True, prog_bar=True, logger=True)
        self.log("test_cindex_simple", cindex, on_epoch=True, prog_bar=True, logger=True)
        self.targets.clear()


    def configure_optimizers(self):
        """
        configures the optimizer and learning rate scheduler:
        * Optimizer: Adam optimizer with weight decay (L2 regularization).
        * Scheduler: Cosine Annealing scheduler, which adjusts the learning rate according to a cosine curve.
        """
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.lr, weight_decay=self.hparams.weight_decay)
        scheduler_config = {
            "scheduler": torch.optim.lr_scheduler.CosineAnnealingLR(
                optimizer,
                T_max=45,
                eta_min=6e-3
            ),
            "interval": "epoch",
            "frequency": 1,
            "strict": False,
        }

        return {"optimizer": optimizer, "lr_scheduler": scheduler_config}

## Train and Predict

In [ ]:
import json
import pytorch_lightning as pl
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import torch
from pytorch_lightning.callbacks import LearningRateMonitor, TQDMProgressBar
from pytorch_lightning.callbacks import StochasticWeightAveraging
from sklearn.model_selection import StratifiedKFold

pl.seed_everything(42)

def main(hparams):
    """
    Main function to train the model.
    The steps are as following :
    * load data and fill efs and efs time for test data with 1
    * initialize pred array with 0
    * get categorical and numerical columns
    * split the train data on the stratified criterion : race_group * newborns yes/no
    * preprocess the fold data (create dataloaders)
    * train the model and create final submission output
    """
    test, train_original = load_data()
    test['efs_time'] = 1
    test['efs'] = 1
    test_pred = np.zeros(test.shape[0])
    nn_oof_preds = np.zeros(train_original.shape[0])
    categorical_cols, numerical = get_feature_types(train_original)
    kf = StratifiedKFold(n_splits=5, shuffle=True, )
    for i, (train_index, test_index) in enumerate(
        kf.split(
            train_original, train_original.race_group.astype(str) + (train_original.age_at_hct == 0.044).astype(str)
        )
    ):
        tt = train_original.copy()
        train = tt.iloc[train_index]
        val = tt.iloc[test_index]
        X_cat_val, X_num_train, X_num_val, dl_train, dl_val, transformers = preprocess_data(train, val)
        model = train_final(X_num_train, dl_train, dl_val, transformers, categorical_cols=categorical_cols)
        val_pred, _ = model.cuda().eval()(
            torch.tensor(X_cat_val, dtype=torch.long).cuda(),
            torch.tensor(X_num_val, dtype=torch.float32).cuda()
        )
        nn_oof_preds[test_index] = val_pred.detach().cpu().numpy()
        # Create submission
        train = tt.iloc[train_index]
        X_cat_val, X_num_train, X_num_val, dl_train, dl_val, transformers = preprocess_data(train, test)
        pred, _ = model.cuda().eval()(
            torch.tensor(X_cat_val, dtype=torch.long).cuda(),
            torch.tensor(X_num_val, dtype=torch.float32).cuda()
        )
        test_pred += pred.detach().cpu().numpy()
    test_pred /= 5  
    # subm_data = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/sample_submission.csv")
    # subm_data['prediction'] = -test_pred
    # subm_data.to_csv('submission.csv', index=False)
    
    # display(subm_data.head())
    return test_pred, nn_oof_preds



def train_final(X_num_train, dl_train, dl_val, transformers, hparams=None, categorical_cols=None):
    """
    Defines model hyperparameters and fit the model.
    """
    if hparams is None:
        hparams = {
            "embedding_dim": 16,
            "projection_dim": 112,
            "hidden_dim": 56,
            "lr": 0.06464861983337984,
            "dropout": 0.05463240181423116,
            "aux_weight": 0.26545778308743806,
            "margin": 0.2588153271003354,
            "weight_decay": 0.0002773544957610778
        }
    model = LitNN(
        continuous_dim=X_num_train.shape[1],
        categorical_cardinality=[len(t.classes_) for t in transformers],
        race_index=categorical_cols.index("race_group"),
        **hparams
    )
    checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor="val_loss", save_top_k=1)
    trainer = pl.Trainer(
        accelerator='cuda',
        max_epochs=70,
        log_every_n_steps=6,
        callbacks=[
            checkpoint_callback,
            LearningRateMonitor(logging_interval='epoch'),
            TQDMProgressBar(),
            StochasticWeightAveraging(swa_lrs=1e-5, swa_epoch_start=45, annealing_epochs=15)
        ],
    )
    trainer.fit(model, dl_train)
    trainer.test(model, dl_val)
    return model.eval()


hparams = None
nn_preds, nn_oof_preds = main(hparams)
print("done")

# GBDTs

## Prepare and Config

In [ ]:
import warnings
from pathlib import Path
import numpy as np
import pandas as pd
import polars as pl
pd.options.display.max_columns = None
warnings.filterwarnings('ignore')
from joblib import dump, load

# lifelines
from lifelines import CoxPHFitter
from lifelines import KaplanMeierFitter
from lifelines import NelsonAalenFitter

# for models
import lightgbm as lgb
from scipy.stats import rankdata 
from catboost import CatBoostRegressor
import xgboost as xgb
from sklearn.model_selection import KFold, StratifiedKFold

"""
To evaluate the equitable prediction of transplant survival outcomes,
we use the concordance index (C-index) between a series of event
times and a predicted score across each race group.
 
It represents the global assessment of the model discrimination power:
this is the model’s ability to correctly provide a reliable ranking
of the survival times based on the individual risk scores.
 
The concordance index is a value between 0 and 1 where:
 
0.5 is the expected result from random predictions,
1.0 is perfect concordance (with no censoring, otherwise <1.0),
0.0 is perfect anti-concordance (with no censoring, otherwise >0.0)

"""

import pandas as pd
import pandas.api.types
import numpy as np
from lifelines.utils import concordance_index
from colorama import Fore, Back, Style

class ParticipantVisibleError(Exception):
    pass


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    """
    >>> import pandas as pd
    >>> row_id_column_name = "id"
    >>> y_pred = {'prediction': {0: 1.0, 1: 0.0, 2: 1.0}}
    >>> y_pred = pd.DataFrame(y_pred)
    >>> y_pred.insert(0, row_id_column_name, range(len(y_pred)))
    >>> y_true = { 'efs': {0: 1.0, 1: 0.0, 2: 0.0}, 'efs_time': {0: 25.1234,1: 250.1234,2: 2500.1234}, 'race_group': {0: 'race_group_1', 1: 'race_group_1', 2: 'race_group_1'}}
    >>> y_true = pd.DataFrame(y_true)
    >>> y_true.insert(0, row_id_column_name, range(len(y_true)))
    >>> score(y_true.copy(), y_pred.copy(), row_id_column_name)
    0.75
    """
    
    del solution[row_id_column_name]
    del submission[row_id_column_name]
    
    event_label = 'efs'
    interval_label = 'efs_time'
    prediction_label = 'prediction'
    for col in submission.columns:
        if not pandas.api.types.is_numeric_dtype(submission[col]):
            raise ParticipantVisibleError(f'Submission column {col} must be a number')
    # Merging solution and submission dfs on ID
    merged_df = pd.concat([solution, submission], axis=1)
    merged_df.reset_index(inplace=True)
    merged_df_race_dict = dict(merged_df.groupby(['race_group']).groups)
    metric_list = []
    for race in merged_df_race_dict.keys():
        # Retrieving values from y_test based on index
        indices = sorted(merged_df_race_dict[race])
        merged_df_race = merged_df.iloc[indices]
        # Calculate the concordance index
        c_index_race = concordance_index(
                        merged_df_race[interval_label],
                        -merged_df_race[prediction_label],
                        merged_df_race[event_label])
        metric_list.append(c_index_race)
    return float(np.mean(metric_list)-np.sqrt(np.var(metric_list)))


In [ ]:
class Config:
    if not kaggle:
        train_path = '../data/train.csv'
        test_path = '../data/test.csv'
        subm_path = '../data/sample_submission.csv'
    else:
        train_path = Path('/kaggle/input/equity-post-HCT-survival-predictions/train.csv')
        test_path = Path('/kaggle/input/equity-post-HCT-survival-predictions/test.csv')
        subm_path = Path('/kaggle/input/equity-post-HCT-survival-predictions/sample_submission.csv')


    early_stopping_round = 300

    batch_size = 32768
    early_stop = 300
    penalizer = 0.01
    n_splits = 10
    seed = 42

    weights = [1.0, 1.0, 8.0, 4.0, 8.0, 4.0, 6.0, 6.0]
    # weights = [0.0, 0.0, 8.0, 4.0, 8.0, 4.0, 6.0, 6.0]

    ctb_params = {
        'loss_function': 'RMSE',
        'learning_rate': 0.03,
        'random_state': 42,
        'task_type': 'CPU',
        'num_trees': 6000,
        'subsample': 0.85,
        'reg_lambda': 8.0,
        'depth': 8,
        # 'thread_count': 12,
    }

    lgb_params = {
        'objective': 'regression',
        'min_child_samples': 32,
        'num_iterations': 6000,
        'learning_rate': 0.03,
        'extra_trees': True,
        'reg_lambda': 8.0,
        'reg_alpha': 0.1,
        'num_leaves': 64,
        'metric': 'rmse',
        'max_depth': 8,
        'device': 'cpu',
        'max_bin': 128,
        'verbose': -1,
        'seed': 42
    }

    # Parameters for the first CatBoost model with Cox loss function
    cox1_params = {
        'grow_policy': 'Depthwise',
        'min_child_samples': 8,
        'loss_function': 'Cox',
        'learning_rate': 0.03,
        'random_state': 42,
        'task_type': 'CPU',
        'num_trees': 6000,
        'subsample': 0.85,
        'reg_lambda': 8.0,
        'depth': 8
    }

    # Parameters for the second CatBoost model with Cox loss function
    cox2_params = {
        'grow_policy': 'Lossguide',
        'loss_function': 'Cox',
        'learning_rate': 0.03,
        'random_state': 42,
        'task_type': 'CPU',
        'num_trees': 6000,
        'subsample': 0.85,
        'reg_lambda': 8.0,
        'num_leaves': 32,
        'depth': 8
    }

## Feature Engineering

In [ ]:
class FeatureEngineer:

    def __init__(self, batch_size):
        self._batch_size = batch_size

    def load_data(self, path):

        return pl.read_csv(path, batch_size=self._batch_size)

    def cast_datatypes(self, df):

        num_cols = [ 'hla_high_res_8', 'hla_low_res_8', 'hla_high_res_6', 'hla_low_res_6', 'hla_high_res_10', 'hla_low_res_10', 'hla_match_dqb1_high', 'hla_match_dqb1_low', 'hla_match_drb1_high', 'hla_match_drb1_low', 'hla_nmdp_6', 'year_hct', 'hla_match_a_high', 'hla_match_a_low', 'hla_match_b_high', 'hla_match_b_low', 'hla_match_c_high', 'hla_match_c_low', 'donor_age', 'age_at_hct', 'comorbidity_score', 'karnofsky_score', 'efs', 'efs_time' ]

        # fill null values
        for col in df.columns:
            if col in num_cols:
                # df = df.with_columns(pl.col(col).fill_null(-1).cast(pl.Float32))  
                df = df.with_columns(pl.col(col).cast(pl.Float32))  
            else:
                df = df.with_columns(pl.col(col).fill_null('Unknown').cast(pl.String))  

        return df.with_columns(pl.col('ID').cast(pl.Int32))

    def recalculate_hla_sums(self, df):
        
        df = df.with_columns(
            (pl.col("hla_match_a_low").fill_null(0) + pl.col("hla_match_b_low").fill_null(0) + 
             pl.col("hla_match_drb1_high").fill_null(0)).alias("hla_nmdp_6"),
            
            (pl.col("hla_match_a_low").fill_null(0) + pl.col("hla_match_b_low").fill_null(0) + 
             pl.col("hla_match_drb1_low").fill_null(0)).alias("hla_low_res_6"),
            
            (pl.col("hla_match_a_high").fill_null(0) + pl.col("hla_match_b_high").fill_null(0) + 
             pl.col("hla_match_drb1_high").fill_null(0)).alias("hla_high_res_6"),
            
            (pl.col("hla_match_a_low").fill_null(0) + pl.col("hla_match_b_low").fill_null(0) + 
             pl.col("hla_match_c_low").fill_null(0) + pl.col("hla_match_drb1_low").fill_null(0)
            ).alias("hla_low_res_8"),
            
            (pl.col("hla_match_a_high").fill_null(0) + pl.col("hla_match_b_high").fill_null(0) + 
             pl.col("hla_match_c_high").fill_null(0) + pl.col("hla_match_drb1_high").fill_null(0)
            ).alias("hla_high_res_8"),
            
            (pl.col("hla_match_a_low").fill_null(0) + pl.col("hla_match_b_low").fill_null(0) + 
             pl.col("hla_match_c_low").fill_null(0) + pl.col("hla_match_drb1_low").fill_null(0) +
             pl.col("hla_match_dqb1_low").fill_null(0)).alias("hla_low_res_10"),
            
            (pl.col("hla_match_a_high").fill_null(0) + pl.col("hla_match_b_high").fill_null(0) + 
             pl.col("hla_match_c_high").fill_null(0) + pl.col("hla_match_drb1_high").fill_null(0) +
             pl.col("hla_match_dqb1_high").fill_null(0)).alias("hla_high_res_10"),
        )

        return df
    

    def numeric_fe(self, df):
        # df['num_null_count'] = (df.isna()).sum(axis=1)
        # df['total_null_count'] = df['num_null_count'] + df['cat_null_count']
        # df['age_diff'] = abs(df['donor_age'] - df['age_at_hct'])
        df['age_diff'] = df['donor_age'] - df['age_at_hct']
        df['age_ratio'] = df['donor_age'] / df['age_at_hct']
        # df.loc[((df.donor_age < 0) | (df.age_at_hct < 0)),'age_diff'] = -1
        # df.loc[((df.donor_age < 0) | (df.age_at_hct < 0)),'age_ratio'] = -1
        # df['older_donor'] = df['age_ratio'].apply(lambda x: 'Yes' if x>1 else 'No')
        # df.loc[((df['donor_age'].isna())|(df['age_at_hct'].isna())),'older_donor'] = 'Unknown'
        # df['null_count_diff'] = df['cat_null_count'] - df['num_null_count']

        return df

    def cat_fe(self, df):
        # df['cat_null_count'] = (df=="Unknown").sum(axis=1)

        return df

    def select_features(self, df):
        base_features = ['ID', 'dri_score', 'psych_disturb', 'cyto_score', 'diabetes',
            'hla_match_c_high', 'hla_high_res_8', 'tbi_status', 'arrhythmia',
            'hla_low_res_6', 'graft_type', 'vent_hist', 'renal_issue',
            'pulm_severe', 'prim_disease_hct', 'hla_high_res_6', 'cmv_status',
            'hla_high_res_10', 'hla_match_dqb1_high', 'tce_imm_match', 'hla_nmdp_6',
            'hla_match_c_low', 'rituximab', 'hla_match_drb1_low',
            'hla_match_dqb1_low', 'prod_type', 'cyto_score_detail',
            'conditioning_intensity', 'ethnicity', 'year_hct', 'obesity', 'mrd_hct',
            'in_vivo_tcd', 'tce_match', 'hla_match_a_high', 'hepatic_severe',
            'donor_age', 'prior_tumor', 'hla_match_b_low', 'peptic_ulcer',
            'age_at_hct', 'hla_match_a_low', 'gvhd_proph', 'rheum_issue',
            'sex_match', 'hla_match_b_high', 'race_group', 'comorbidity_score',
            'karnofsky_score', 'hepatic_mild', 'tce_div_match', 'donor_related',
            'melphalan_dose', 'hla_low_res_8', 'cardiac', 'hla_match_drb1_high',
            'pulm_moderate', 'hla_low_res_10', 'efs', 'efs_time']

        selected_features = ['older_donor', 'total_null_count', 'null_count_diff', 'age_ratio'] 
        features = list(set(df.columns) & set(base_features + selected_features))
        return df[features]

    def info(self, df):
        
        print(f'\nShape of dataframe: {df.shape}') 
        
        mem = df.memory_usage().sum() / 1024**2
        print('Memory usage: {:.2f} MB\n'.format(mem))

        display(df.head())

    def apply_fe(self, path):

        df = self.load_data(path)
        df = self.cast_datatypes(df)
        df = self.recalculate_hla_sums(df)
        df = df.to_pandas()
        df = self.cat_fe(df)
        df = self.numeric_fe(df)

        df = self.select_features(df)

        self.info(df)
        
        cat_cols = [col for col in df.columns if df[col].dtype == pl.String]
        print(cat_cols)

        return df, cat_cols

In [ ]:
feature_engineer = FeatureEngineer(Config.batch_size)
train_data, cat_cols = feature_engineer.apply_fe(Config.train_path)
test_data, cat_cols = feature_engineer.apply_fe(Config.test_path)

## Model

### define targets

In [ ]:
class Targets:

    def __init__(self, data, cat_cols, penalizer, n_splits):
        
        self.data = data
        self.cat_cols = cat_cols
        
        self._length = len(self.data)
        self._penalizer = penalizer
        self._n_splits = n_splits

    def _prepare_cv(self):
        
        oof_preds = np.zeros(self._length)
            
        cv = KFold(n_splits=self._n_splits, shuffle=True, random_state=42)

        return cv, oof_preds

    def validate_model(self, preds, title):
            
        y_true = self.data[['ID', 'efs', 'efs_time', 'race_group']].copy()
        y_pred = self.data[['ID']].copy()
        
        y_pred['prediction'] = preds
            
        c_index_score = score(y_true.copy(), y_pred.copy(), 'ID')
        print(f'Overall Stratified C-Index Score for {title}: {c_index_score:.4f}')
        return c_index_score
    def create_target1(self):  

        '''
        Inside the CV loop, constant columns are dropped if they exist in a fold. Otherwise, the code produces error:

        delta contains nan value(s). Convergence halted. Please see the following tips in the lifelines documentation: 
        https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-model
        '''

        cv, oof_preds = self._prepare_cv()

        # Apply one hot encoding to categorical columns
        data = pd.get_dummies(self.data, columns=self.cat_cols, drop_first=True).drop('ID', axis=1) 
        data = data.fillna(-1)

        for train_index, valid_index in cv.split(data):

            train_data = data.iloc[train_index]
            valid_data = data.iloc[valid_index]

            # Drop constant columns if they exist
            train_data = train_data.loc[:, train_data.nunique() > 1]
            valid_data = valid_data[train_data.columns]

            cph = CoxPHFitter(penalizer=self._penalizer)
            cph.fit(train_data, duration_col='efs_time', event_col='efs')
            
            oof_preds[valid_index] = cph.predict_partial_hazard(valid_data)              

        self.data['target1'] = oof_preds 
        self.validate_model(oof_preds, 'Cox') 

        return self.data

    def create_target2(self):        

        cv, oof_preds = self._prepare_cv()

        for train_index, valid_index in cv.split(self.data):

            train_data = self.data.iloc[train_index]
            valid_data = self.data.iloc[valid_index]

            kmf = KaplanMeierFitter()
            kmf.fit(durations=train_data['efs_time'], event_observed=train_data['efs'])
            
            oof_preds[valid_index] = kmf.survival_function_at_times(valid_data['efs_time']).values

        self.data['target2'] = oof_preds  
        self.validate_model(oof_preds, 'Kaplan-Meier')

        return self.data

    def create_target3(self):        

        cv, oof_preds = self._prepare_cv()

        for train_index, valid_index in cv.split(self.data):

            train_data = self.data.iloc[train_index]
            valid_data = self.data.iloc[valid_index]
            
            naf = NelsonAalenFitter()
            naf.fit(durations=train_data['efs_time'], event_observed=train_data['efs'])
            
            oof_preds[valid_index] = -naf.cumulative_hazard_at_times(valid_data['efs_time']).values

        self.data['target3'] = oof_preds  
        self.validate_model(oof_preds, 'Nelson-Aalen')

        return self.data

    def create_target4(self):

        self.data['target4'] = self.data.efs_time.copy()
        self.data.loc[self.data.efs == 0, 'target4'] *= -1

        return self.data

### define model

In [ ]:
from catboost import  Pool
class Model:
    
    def __init__(self, data, cat_cols, early_stop, penalizer, n_splits):
        
        self.targets = Targets(data, cat_cols, penalizer, n_splits)
        
        self.data = data
        self.cat_cols = ['peptic_ulcer', 'ethnicity', 'prim_disease_hct', 'tbi_status', 'tce_div_match', 'donor_related', 'melphalan_dose', 'diabetes', 'pulm_severe', 'cardiac', 'obesity', 'rheum_issue', 'gvhd_proph', 'graft_type', 'cmv_status', 'renal_issue', 'sex_match', 'race_group', 'tce_imm_match', 'cyto_score_detail', 'arrhythmia', 'mrd_hct', 'pulm_moderate', 'hepatic_severe', 'dri_score', 'in_vivo_tcd', 'conditioning_intensity', 'rituximab', 'prior_tumor', 'hepatic_mild', 'psych_disturb', 'tce_match', 'prod_type', 'cyto_score', 'vent_hist']
        self._early_stop = early_stop

    def create_targets(self):
        print('creating target 1')
        self.data = self.targets.create_target1()
        print('creating target 2')
        self.data = self.targets.create_target2()
        print('creating target 3')
        self.data = self.targets.create_target3()
        print('creating target 4')
        self.data = self.targets.create_target4()

        return self.data
        
    def train_model(self, params, target, title):
        
        for col in self.cat_cols:
            self.data[col] = self.data[col].astype('category')
            
        X = self.data.drop(['ID', 'efs', 'efs_time', 'target1', 'target2', 'target3', 'target4'], axis=1)
        y = self.data[target]
        
        models, fold_scores = [], []
            
        cv, oof_preds = self.targets._prepare_cv()
    
        for fold, (train_index, valid_index) in enumerate(cv.split(X, y)):
                
            X_train = X.iloc[train_index]
            X_valid = X.iloc[valid_index]
                
            y_train = y.iloc[train_index]
            y_valid = y.iloc[valid_index]
    
            if title.startswith('LightGBM'):
                        
                model = lgb.LGBMRegressor(**params)
                        
                model.fit(
                    X_train, 
                    y_train,  
                    eval_set=[(X_valid, y_valid)],
                    eval_metric='rmse',
                    callbacks=[lgb.early_stopping(self._early_stop, verbose=0), lgb.log_evaluation(0)]
                )
                data = X_valid        
            elif title.startswith('CatBoost'):
                # feature selection
                # X_train = X_train.drop(columns=['age_diff'])
                # X_valid = X_valid.drop(columns=['age_diff'])
                model = CatBoostRegressor(**params, verbose=0, cat_features=self.cat_cols)
                # model.fit(
                #     X_train,
                #     y_train,
                #     eval_set=(X_valid, y_valid),
                #     early_stopping_rounds=self._early_stop, 
                #     verbose=0
                # )               
                # model.save_model(f'./checkpoints/baseline/catboost_{fold}_{target}_{title}.cbm')

                if kaggle:
                    model.load_model(f'/kaggle/input/baseline-naive-post-processing/catboost_{fold}_{target}_{title}.cbm')
                else:
                    model.load_model(f'./checkpoints/baseline/catboost_{fold}_{target}_{title}.cbm')
                
                data = Pool(data=X_valid, cat_features=self.cat_cols)                 
            models.append(model)
            
            oof_preds[valid_index] = model.predict(data)

            y_true_fold = self.data.iloc[valid_index][['ID', 'efs', 'efs_time', 'race_group']].copy()
            y_pred_fold = self.data.iloc[valid_index][['ID']].copy()
            
            y_pred_fold['prediction'] = oof_preds[valid_index]
    
            fold_score = score(y_true_fold, y_pred_fold, 'ID')
            fold_scores.append(fold_score)
    
        self.targets.validate_model(oof_preds, title)
        
        return models, oof_preds

    def infer_model(self, data, models, title):
        
        data = data.drop(['ID'], axis=1)
        # if title.startswith('CatBoost'):
            # data = data.drop(['age_diff'], axis=1)

        for col in self.cat_cols:
            data[col] = data[col].astype('category')
        if title.startswith('LightGBM'):
            return np.mean([model.predict(data) for model in models], axis=0)
        elif title.startswith('CatBoost'):
            data_ = Pool(data=data, cat_features=self.cat_cols)   
            return np.mean([model.predict(data_) for model in models], axis=0)

### create targets

In [ ]:
cat_cols = ['peptic_ulcer', 'ethnicity', 'prim_disease_hct', 'tbi_status', 'tce_div_match', 'donor_related', 'melphalan_dose', 'diabetes', 'pulm_severe', 'cardiac', 'obesity', 'rheum_issue', 'gvhd_proph', 'graft_type', 'cmv_status', 'renal_issue', 'sex_match', 'race_group', 'tce_imm_match', 'cyto_score_detail', 'arrhythmia', 'mrd_hct', 'pulm_moderate', 'hepatic_severe', 'dri_score', 'in_vivo_tcd', 'conditioning_intensity', 'rituximab', 'prior_tumor', 'hepatic_mild', 'psych_disturb', 'tce_match', 'prod_type', 'cyto_score', 'vent_hist']
model = Model(data=train_data, cat_cols=cat_cols, early_stop=Config.early_stopping_round, penalizer=Config.penalizer, n_splits=Config.n_splits)
train_data = model.create_targets()

### train and preds

In [ ]:
# Cox target
ctb1_models, ctb1_oof_preds = model.train_model(Config.ctb_params, target='target1', title='CatBoost')
lgb1_models, lgb1_oof_preds = model.train_model(Config.lgb_params, target='target1', title='LightGBM')
ctb1_preds = model.infer_model(test_data, ctb1_models, title='CatBoost')
lgb1_preds = model.infer_model(test_data, lgb1_models, title='LightGBM')
# Kaplan Meier target
ctb2_models, ctb2_oof_preds = model.train_model(Config.ctb_params, target='target2', title='CatBoost')
lgb2_models, lgb2_oof_preds = model.train_model(Config.lgb_params, target='target2', title='LightGBM')
ctb2_preds = model.infer_model(test_data, ctb2_models, title='CatBoost')
lgb2_preds = model.infer_model(test_data, lgb2_models, title='LightGBM')
# Nelson Aalen target
ctb3_models, ctb3_oof_preds = model.train_model(Config.ctb_params, target='target3', title='CatBoost')
lgb3_models, lgb3_oof_preds = model.train_model(Config.lgb_params, target='target3', title='LightGBM')
ctb3_preds = model.infer_model(test_data, ctb3_models, title='CatBoost')
lgb3_preds = model.infer_model(test_data, lgb3_models, title='LightGBM')
# Cox-loss target
cox1_models, cox1_oof_preds = model.train_model(Config.cox1_params, target='target4', title='CatBoost')
cox2_models, cox2_oof_preds = model.train_model(Config.cox2_params, target='target4', title='CatBoost')
cox1_preds = model.infer_model(test_data, cox1_models, title='CatBoost')
cox2_preds = model.infer_model(test_data, cox2_models, title='CatBoost')

## Classifier for post-processing

In [ ]:

from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier, early_stopping, log_evaluation
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

n_splits = Config.n_splits
seed = Config.seed
cv = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
n_train = len(train_data)

target_cols = [col for col in train_data.columns if col.startswith('target')]
drop_cols = ['ID', 'efs', 'efs_time'] + target_cols
features = train_data.drop(columns=drop_cols).columns

# =============================================================================
# Level 1: LGB, XGB, CTB
# =============================================================================

oof_lgb = np.zeros(n_train)
oof_xgb = np.zeros(n_train)
oof_ctb = np.zeros(n_train)

cls_preds_lgb = []
cls_preds_xgb = []
cls_preds_ctb = []

for fold, (train_idx, valid_idx) in enumerate(cv.split(train_data), 1):
    print(f"====== Level1 - Fold {fold} ======")
    
    train_fold = train_data.iloc[train_idx].copy()
    valid_fold = train_data.iloc[valid_idx].copy()
    
    X_train = train_fold.drop(columns=drop_cols).copy()
    y_train = train_fold['efs']
    X_valid = valid_fold.drop(columns=drop_cols).copy()
    y_valid = valid_fold['efs']
    
    for col in cat_cols:
        if col in X_train.columns:
            X_train[col] = X_train[col].astype('category')
        if col in X_valid.columns:
            X_valid[col] = X_valid[col].astype('category')

    # LightGBM
    model_lgb = LGBMClassifier(
        objective='binary',
        boosting_type='gbdt',
        max_depth=3,
        learning_rate=0.02,
        n_estimators=5000,
        random_state=seed
    )
    model_lgb.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        callbacks=[early_stopping(300, verbose=0), log_evaluation(0)],
        categorical_feature=cat_cols
    )
    preds_lgb = model_lgb.predict_proba(X_valid)[:, 1]
    oof_lgb[valid_idx] = preds_lgb
    
    X_test = test_data[features].copy()
    for col in cat_cols:
        if col in X_test.columns:
            X_test[col] = X_test[col].astype('category')
    cls_pred_lgb = model_lgb.predict_proba(X_test)[:, 1]
    cls_preds_lgb.append(cls_pred_lgb)
    
    # XGBoost
    model_xgb = xgb.XGBClassifier(
        objective='binary:logistic',
        eval_metric='auc',
        use_label_encoder=False,
        max_depth=3,
        learning_rate=0.02,
        n_estimators=5000,
        random_state=seed,
        enable_categorical=True,
        early_stopping_rounds=300,
        verbose=False
    )
    model_xgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)
    preds_xgb = model_xgb.predict_proba(X_valid)[:, 1]
    oof_xgb[valid_idx] = preds_xgb
    
    cls_pred_xgb = model_xgb.predict_proba(X_test)[:, 1]
    cls_preds_xgb.append(cls_pred_xgb)
    
    # CatBoost
    model_ctb = CatBoostClassifier(
        iterations=5000,
        learning_rate=0.02,
        depth=3,
        eval_metric='AUC',
        random_seed=seed,
        od_type='Iter',
        od_wait=300,
        verbose=False
    )
    model_ctb.fit(
        X_train, y_train,
        eval_set=(X_valid, y_valid),
        cat_features=cat_cols,
        use_best_model=True,
        verbose=False
    )
    preds_ctb = model_ctb.predict_proba(X_valid)[:, 1]
    oof_ctb[valid_idx] = preds_ctb
    
    cls_pred_ctb = model_ctb.predict_proba(X_test)[:, 1]
    cls_preds_ctb.append(cls_pred_ctb)
    
    auc_lgb_fold = roc_auc_score(y_valid, preds_lgb)
    auc_xgb_fold = roc_auc_score(y_valid, preds_xgb)
    auc_ctb_fold = roc_auc_score(y_valid, preds_ctb)
    print(f"Fold {fold} AUC - LightGBM: {auc_lgb_fold:.4f}, XGBoost: {auc_xgb_fold:.4f}, CatBoost: {auc_ctb_fold:.4f}")

auc_lgb = roc_auc_score(train_data['efs'], oof_lgb)
auc_xgb = roc_auc_score(train_data['efs'], oof_xgb)
auc_ctb = roc_auc_score(train_data['efs'], oof_ctb)
print("----- Overall Base Model CV Scores -----")
print(f"LightGBM: {auc_lgb:.4f}")
print(f"XGBoost:  {auc_xgb:.4f}")
print(f"CatBoost: {auc_ctb:.4f}")

X_meta_train = np.column_stack((oof_lgb, oof_xgb, oof_ctb))
y_meta = train_data['efs'].values

cls_pred_lgb_final = np.mean(cls_preds_lgb, axis=0)
cls_pred_xgb_final = np.mean(cls_preds_xgb, axis=0)
cls_pred_ctb_final = np.mean(cls_preds_ctb, axis=0)
X_meta_test = np.column_stack((cls_pred_lgb_final, cls_pred_xgb_final, cls_pred_ctb_final))

# =============================================================================
# Level 2: Stacking with Logistic Regression
# =============================================================================

cls_oof_preds = np.zeros(n_train)
test_preds_meta_folds = []

for fold, (meta_train_idx, meta_valid_idx) in enumerate(cv.split(X_meta_train), 1):
    print(f"====== Level2 (LR) - Fold {fold} ======")
    X_meta_tr = X_meta_train[meta_train_idx]
    y_meta_tr = y_meta[meta_train_idx]
    X_meta_val = X_meta_train[meta_valid_idx]
    
    lr_meta = LogisticRegression(max_iter=1000, random_state=seed)
    lr_meta.fit(X_meta_tr, y_meta_tr)
    
    preds_meta_val = lr_meta.predict_proba(X_meta_val)[:, 1]
    cls_oof_preds[meta_valid_idx] = preds_meta_val
    
    preds_meta_test = lr_meta.predict_proba(X_meta_test)[:, 1]
    test_preds_meta_folds.append(preds_meta_test)

meta_cv_auc = roc_auc_score(y_meta, cls_oof_preds)
print("Level2 (LR) CV AUC:", meta_cv_auc)

# 最終テスト予測：各fold のメタモデル予測の平均
cls_preds = np.mean(test_preds_meta_folds, axis=0)


# Ensembling

### optimize weights

In [ ]:
# import numpy as np
# import pandas as pd
# from scipy.stats import rankdata

# # Supondo que as previsões fora da amostra (oof_preds) já estejam definidas
# oof_preds = [
#     ctb1_oof_preds, 
#     lgb1_oof_preds, 
#     ctb2_oof_preds, 
#     lgb2_oof_preds, 
#     ctb3_oof_preds, 
#     lgb3_oof_preds, 
#     cox1_oof_preds,
#     cox2_oof_preds,
#     cls_oof_preds,
#     tabm_oof_preds,
#     nn_oof_preds
# ]

# df_oof = pd.DataFrame(np.array(oof_preds).T, columns=['ctb1', 'lgb1', 'ctb2', 'lgb2', 'ctb3', 'lgb3', 'cox1', 'cox2', 'cls', 'tabm', 'nn'])
# df_oof['tabm'] = -1*df_oof['tabm'] 
# for col in ['ctb1', 'lgb1', 'ctb2', 'lgb2', 'ctb3', 'lgb3', 'cox1', 'cox2','tabm', 'nn']:
#     if col in ['ctb2', 'lgb2', 'ctb3', 'lgb3', 'tabm', 'nn']:
#         df_oof.loc[df_oof.cls < 0.4, col] -= 0.01
#         df_oof.loc[df_oof.cls < 0.3, col] -= 0.05
#         df_oof.loc[df_oof.cls < 0.2, col] -= 0.03
#         df_oof.loc[df_oof.cls < 0.1, col] -= 0.02
#     else:
#         df_oof.loc[df_oof.cls < 0.4, col] -= 0.1
#         df_oof.loc[df_oof.cls < 0.3, col] -= 0.5
#         df_oof.loc[df_oof.cls < 0.2, col] -= 0.3
#         df_oof.loc[df_oof.cls < 0.1, col] -= 0.2

#     df_oof[col] = rankdata(df_oof[col].values)

# ranked_oof_preds = df_oof.drop(columns=['cls']).values

# # Função para avaliar o modelo
# def evaluate_model(weights, ranked_oof_preds):
#     ensemble_oof_preds = np.dot(weights, ranked_oof_preds.T)
#     return model.targets.validate_model(ensemble_oof_preds, 'Ensemble Model')

# # Loop para testar pesos randomicamente
# best_score = 0.00000000
# best_weights = None

# for _ in range(5000):  # Número de iterações para testar diferentes pesos
#     weights = np.random.randint(1, 10, size=ranked_oof_preds.shape[1])
#     score_ = evaluate_model(weights, ranked_oof_preds)
#     if score_ > best_score:
#         best_score = score_
#         best_weights = weights
#         print(f'Overall Stratified C-Index Score for {_}: {best_score:.4f}')


# print(f"Best weights: {best_weights}")
# print(f"Best score: {best_score}")

# # Usar os melhores pesos encontrados
# ensemble_oof_preds = np.dot(best_weights, ranked_oof_preds.T)
# model.targets.validate_model(ensemble_oof_preds, 'Ensemble Model')

### eval and test preds

In [ ]:
# tuned parameters by optuna
post_processing_params = {
    'target1': {'alpha': 2.3737589590478634, 'beta': 0.5446072498650629},
    'target2': {'alpha': 0.49950486318733067, 'beta': 0.5535139507972653},
    'target3': {'alpha': 0.6716850491630524, 'beta': 0.5513477964875512},
    'cox': {'alpha': 0.5121262482791129, 'beta': 0.3760593640066433},
    'nn': {'alpha': 0.5568961571370223, 'beta': 0.5350792198430299}
}
post_processing_params_2 = {
    'target1': {'alpha': 2.133612144161301, 'beta': 0.5358471298308779, 'theta': 4.97486323269561, 'gamma': 0.3606124701773922},
    'target2': {'alpha': 2.8916213733568434, 'beta': 0.07778082208568068, 'theta': 0.5594454604979927, 'gamma': 0.5508674763236482},
    'target3': {'alpha': 8.053979380959879, 'beta': 0.04968448715170445, 'theta': 0.7959195008259203, 'gamma': 0.5439370628940968},
    'cox': {'alpha': 0.2015329521236918, 'beta': 0.5033412611203962, 'theta': 2.4161253081245166, 'gamma': 0.00025474963771038306},
    'nn': {'alpha': 0.5231177193181507, 'beta': 0.552686519827984, 'theta': 9.643564315722779, 'gamma': 0.1935930800485984}
}
post_processing_params_3 = {
    'target1': {'alpha': 0.5438125860047218, 'beta': 0.3972615117501584, 'theta': 2.187211170953937, 'gamma': 0.5556612377142085},
    'target2': {'alpha': 6.76894533776031, 'beta': 0.06622910046351782, 'theta': 0.3115416085214386, 'gamma': 0.5271933700524203},
    'target3': {'alpha': 0.3702304550589247, 'beta': 0.5272126628874823, 'theta': 1.2234722537496525, 'gamma': 0.0510497336336084},
    'cox': {'alpha': 2.771838870769918, 'beta': 0.49053715531993836, 'theta': 0.592656682285439, 'gamma': 0.03273674674406374},
    'nn': {'alpha': 0.5231177193181507, 'beta': 0.552686519827984, 'theta': 9.643564315722779, 'gamma': 0.1935930800485984}
}
# best_weights = [1, 1, 9, 4, 5, 1, 1, 1, 9, 7] # local実験より
# best_weights = [1,1,10,8,10,9,1,1,8,10]
# best_weights = [1.79256891e-03, 1.40665835e-03, 8.76313857, 9.13724199,
#  6.14993933, 1.94172161, 3.07829079e-03, 1.92536362e-04,
#  8.18358093, 6.48274880] # float optimized
raw_best_weights = [ 0.8053397509758132,  -1.1224518788678595, 5.222789193720539, 1.9698755232102896, 8.425745803919302, 4.175920628470075, -4.2574706572690015, 5.884953879722604, 9.336471011803011,  6.507420683025827]
rank_best_weights = [ 2.0724118964571505,  -0.3752319156671481,  2.437805614887012,  5.066752070752108,  8.722164637815052,  -1.60920113002672,  -6.483433839175567,  2.8524520599587326,  7.878822935011816, 5.951673945758717]
best_weights = [-3.34293772, 9.41208789]

### post processed ranked ensemble * raw ensemble

In [ ]:
oof_preds = [
    ctb1_oof_preds, 
    lgb1_oof_preds, 
    ctb2_oof_preds, 
    lgb2_oof_preds, 
    ctb3_oof_preds, 
    lgb3_oof_preds, 
    cox1_oof_preds,
    cox2_oof_preds,
    cls_oof_preds,
    -tabm_oof_preds,
    -nn_oof_preds
]
df_oof = pd.DataFrame(np.array(oof_preds).T, columns = ['ctb1','lgb1','ctb2','lgb2','ctb3','lgb3','cox1','cox2','cls', 'tabm', 'nn'])

raw_oof_preds = np.dot(raw_best_weights, df_oof.drop(columns=['cls']).T)

for col in ['ctb1', 'lgb1','ctb2', 'lgb2', 'ctb3', 'lgb3', 'cox1', 'cox2', 'tabm', 'nn']:
    if col in ['ctb1', 'lgb1']:
        p = post_processing_params_3['target1']
    elif col in ['ctb2', 'lgb2']:
        p = post_processing_params_3['target2']
    elif col in ['ctb3', 'lgb3']:
        p = post_processing_params_3['target3']
    elif col in ['cox1', 'cox2']:
        p = post_processing_params_3['cox']
    elif col in ['nn', 'tabm']:
        p = post_processing_params_3['nn']
        
    # df_oof[col] = rankdata(df_oof[col] - np.clip(p['alpha'] * (p['beta'] - df_oof['cls']), 0, None))
    df_oof[col] = rankdata(df_oof[col] - np.clip(p['alpha'] * (p['beta'] - df_oof['cls']), 0, None)- np.clip(p['theta'] * (p['gamma'] - df_oof['cls']), 0, None))

ranked_oof_preds = np.dot(rank_best_weights, df_oof.drop(columns=['cls']).T)
ensemble_oof_preds = np.dot(best_weights, np.array([raw_oof_preds, ranked_oof_preds]))
model.targets.validate_model(ensemble_oof_preds, 'Ensemble Model')

CV of best LB(.693):
Overall Stratified C-Index Score for Ensemble Model: 0.6853
0.6852939208340065

integrate nn pairwise:
Overall Stratified C-Index Score for Ensemble Model: 0.6856
0.6855697694376882

updated post post-processing:
Overall Stratified C-Index Score for Ensemble Model: 0.6871
0.687086016701936

In [ ]:

preds = [
    ctb1_preds, 
    lgb1_preds, 
    ctb2_preds, 
    lgb2_preds, 
    ctb3_preds, 
    lgb3_preds,
    cox1_preds,
    cox2_preds,
    cls_preds,
    -tabm_preds,
    -nn_preds
]

df_preds = pd.DataFrame(np.array(preds).T, columns = ['ctb1','lgb1','ctb2','lgb2','ctb3','lgb3','cox1','cox2','cls', 'tabm', 'nn'])

raw_preds = np.dot(raw_best_weights, df_preds.drop(columns=['cls']).T)

for col in ['ctb1', 'lgb1','ctb2', 'lgb2', 'ctb3', 'lgb3', 'cox1', 'cox2', 'tabm', 'nn']:
    if col in ['ctb1', 'lgb1']:
        p = post_processing_params_3['target1']
    elif col in ['ctb2', 'lgb2']:
        p = post_processing_params_3['target2']
    elif col in ['ctb3', 'lgb3']:
        p = post_processing_params_3['target3']
    elif col in ['cox1', 'cox2']:
        p = post_processing_params_3['cox']
    elif col in ['nn', 'tabm']:
        p = post_processing_params_3['nn']
        
    # df_preds[col] = rankdata(df_preds[col] - np.clip(p['alpha'] * (p['beta'] - df_preds['cls']), 0, None))
    df_preds[col] = rankdata(df_preds[col] - np.clip(p['alpha'] * (p['beta'] - df_preds['cls']), 0, None)- np.clip(p['theta'] * (p['gamma'] - df_preds['cls']), 0, None))


ranked_preds = np.dot(rank_best_weights, df_preds.drop(columns=['cls']).T)
ensemble_preds = np.dot(best_weights, np.array([raw_preds, ranked_preds]))

subm_data = pd.read_csv(Config.subm_path)
subm_data['prediction'] = ensemble_preds

subm_data.to_csv('submission.csv', index=False)
display(subm_data.head())